In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

tfds.disable_progress_bar()

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric])
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']
train_dataset.element_spec

In [ ]:
for example, label in train_dataset.take(1):
    print('text: ', example.numpy())
    print('label:', label.numpy())

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
for example, label in train_dataset.take(1):
    print('texts: ', example[:3].numpy())
    print()
    print('labels: ', label[:3].numpy())

In [ ]:
VOCAB_SIZE=1000
encoder=tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:50]

In [ ]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

In [ ]:
for n in range(3):
    print("Original: ", example[n].numpy())
    print("Round trip: ", " ".join(vocab[encoded_example[n]]))

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
print([layer.supports_masking for layer in model.layers])

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics were out of this world. I would recommend this movie.')
sample_text

In [ ]:
predictions = model.predict([sample_text])
predictions

In [ ]:
padding = "the "*2000
predictions = model.predict(np.array([sample_text, padding]))
predictions

In [ ]:
model.compile(tf.keras.optimizers.Adam(1e-4), loss=tf.keras.losses.BinaryCrossentropy(True), metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)

In [ ]:
test_loss, test_add = model.evaluate(test_dataset)
print("Test loss: ", test_loss)
print("Test acc: ", test_acc)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1,2,1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1,2,2)
plot_graphs(history, 'loss')
plt.ylim(None, 1)

In [ ]:
sample_text = 'The movie was cool. The animation and the graphics were out of this world. I would recommend this movie.'
predictions = model.predict(np.array([sample_text]))
predictions

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,)),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(tf.keras.optimizers.Adam(1e-4), loss=tf.keras.losses.BinaryCrossentropy(True), metrics='accuracy')

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)
print("Test loss: ", test_loss)
print("Test acc: ", test_acc)

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1,2,1)
plot_graphs(history, 'loss')
plt.ylim(None, 1)
plt.subplot(1,2,2)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)

In [ ]:
sample_text = ('The movie was not good. The animation and the graphics '
               'were terrible. I would not recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions)

In [ ]:
sample_text = ('The movie was good')
predictions = model.predict(np.array([sample_text]))
print(predictions)